# Tool Output Utilization Evaluator

## Objective
This sample demonstrates how to use tool output utilization evaluator on agent data. The supported input formats include:
- simple data such as strings and `dict` describing agent responses;
- user-agent conversations in the form of list of agent messages. 

## Time

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend using `gpt-4o`.    

### Prerequisite
```bash
pip install azure-ai-projects azure-identity openai
```
Set these environment variables with your own values:
1) **AZURE_AI_PROJECT_ENDPOINT** - Your Azure AI project endpoint in format: `https://<account_name>.services.ai.azure.com/api/projects/<project_name>`
2) **AZURE_AI_MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator (e.g., gpt-4o-mini)


The Tool Output Utilization Evaluator assesses how effectively an AI agent utilizes the outputs from tools and whether it accurately incorporates this information into its responses.

Scoring is based on two levels:
1. Pass - The agent effectively utilizes tool outputs and accurately incorporates the information into its response.
2. Fail - The agent fails to properly utilize tool outputs or incorrectly incorporates the information into its response.

The evaluation includes the score, a brief explanation, and a final pass/fail result.

This evaluation focuses on measuring whether the agent properly uses the information returned by tools to provide accurate and complete responses to users.

Tool Output Utilization requires following input:
- Query - This can be a single query or a list of messages(conversation history with agent). The original task request from the user.
- Response - Response from Agent (or any GenAI App). This can be a single text response or a list of messages generated as part of Agent Response.
- Tool Definitions - Tool(s) definition used by Agent to answer the query. Required to understand the context and expected outputs of the tools.


### Initialize Tool Output Utilization Evaluator


In [ ]:
import os
from openai.types.evals.create_eval_jsonl_run_data_source_param import SourceFileContentContent
from pprint import pprint
from agent_utils import run_evaluator

# Get environment variables
deployment_name = os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"]

# Data source configuration (defines the schema for evaluation inputs)
data_source_config = {
    "type": "custom",
    "item_schema": {
        "type": "object",
        "properties": {
            "query": {
                "anyOf": [
                    {"type": "string"},
                    {
                        "type": "array",
                        "items": {
                            "type": "object"
                        }
                    }
                ]
            },
            "response": {
                "anyOf": [
                    {"type": "string"},
                    {
                        "type": "array",
                        "items": {
                            "type": "object"
                        }
                    }
                ]
            },
            "tool_definitions": {
                "anyOf": [
                    {"type": "object"},
                    {
                        "type": "array",
                        "items": {
                            "type": "object"
                        }
                    }
                ]
            }
        },
        "required": ["query", "response"]
    },
    "include_sample_schema": True
}

# Data mapping (maps evaluation inputs to evaluator parameters)
data_mapping = {
    "query": "{{item.query}}",
    "response": "{{item.response}}",
    "tool_definitions": "{{item.tool_definitions}}"
}

# Initialization parameters for the evaluator
initialization_parameters = {
    "deployment_name": deployment_name
}

# Initialize the evaluation_contents list - we'll append all test cases here
evaluation_contents = []

### Samples

#### Evaluating Good Tool Output Utilization

In [ ]:
# Test Case 1: Good tool output utilization
query = "How is the weather in Seattle?"
response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_123",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "partly cloudy", "humidity": "68%", "wind": "8 mph NW"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The current weather in Seattle is partly cloudy with a temperature of 15°C. The humidity is at 68% and there's a light wind from the northwest at 8 mph.",
            }
        ],
    }
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "query": query,
            "response": response,
            "tool_definitions": tool_definitions
        }
    )
)

#### Complex Scenario with Multiple Tool Outputs

In [ ]:
# Test Case 2: Complex scenario with multiple tool outputs
query_complex = "Can you send me an email with weather information for Seattle?"
response_complex = [
    {
        "createdAt": "2025-03-26T17:27:35Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:37Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "14°C", "condition": "rainy", "precipitation": "light rain", "wind": "12 mph SW"}}],
    },
    {
        "createdAt": "2025-03-26T17:27:38Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_iq9RuPxqzykebvACgX8pqRW2",
                "name": "send_email",
                "arguments": {
                    "recipient": "your_email@example.com",
                    "subject": "Weather Information for Seattle",
                    "body": "Current weather in Seattle: 14°C and rainy with light rain. Wind is from the southwest at 12 mph. Stay dry!",
                },
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:41Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "tool_call_id": "call_iq9RuPxqzykebvACgX8pqRW2",
        "role": "tool",
        "content": [
            {"type": "tool_result", "tool_result": {"message": "Email successfully sent to your_email@example.com.", "status": "delivered"}}
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:42Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I have successfully sent you an email with the weather information for Seattle. The email includes the current conditions: 14°C and rainy with light rain, plus wind details. The email was delivered successfully to your inbox.",
            }
        ],
    },
]

tool_definitions_multiple = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "query": query_complex,
            "response": response_complex,
            "tool_definitions": tool_definitions_multiple
        }
    )
)

#### Query as Conversation History

In [ ]:
# Test Case 3: Query as conversation history instead of a single string
query_as_conversation = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can fetch weather information and send emails."
    },
    {
        "role": "user", 
        "content": "Hi, I need to know the weather in Seattle for my trip planning."
    },
    {
        "role": "user",
        "content": "Can you also email me a summary so I can reference it later when packing?"
    }
]

response_conversation = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_123",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "partly cloudy", "humidity": "68%", "wind": "8 mph NW", "forecast": "no rain expected"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_email_456",
                "name": "send_email",
                "arguments": {
                    "recipient": "user@example.com",
                    "subject": "Seattle Weather Summary for Trip Planning",
                    "body": "Weather summary for Seattle: Temperature is 15°C, partly cloudy conditions, humidity at 68%, with northwest winds at 8 mph. No rain is expected, so light layers should be sufficient for your trip."
                },
            }
        ],
    },
    {
        "tool_call_id": "call_email_456",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"message": "Email sent successfully", "delivery_status": "delivered"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "Perfect! I've checked the weather in Seattle for your trip planning. It's currently 15°C and partly cloudy with light northwest winds and no rain expected. I've also sent you an email summary with all these details including packing recommendations based on the weather conditions. The email was delivered successfully, so you can reference it when preparing for your trip.",
            }
        ],
    }
]

tool_definitions_conversation = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "query": query_as_conversation,
            "response": response_conversation,
            "tool_definitions": tool_definitions_conversation
        }
    )
)

#### Example of Poor Tool Output Utilization

In [ ]:
# Test Case 4: Example of poor tool output utilization
query_poor = "How is the weather in Seattle?"
# Poor utilization - agent doesn't use the detailed tool output effectively
poor_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_123",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "partly cloudy", "humidity": "68%", "wind": "8 mph NW", "UV_index": "3", "visibility": "10 km"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The weather in Seattle is okay. It's not too bad outside.",  # Poor utilization of detailed weather data
            }
        ],
    }
]

tool_definitions_poor = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

# This should score poorly due to inadequate use of tool output
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "query": query_poor,
            "response": poor_response,
            "tool_definitions": tool_definitions_poor
        }
    )
)

#### Response as String (str)

In [ ]:
# Test Case 5: Response as string
query_str = "What's the weather like in Seattle?"
# Response as a simple string (not a list of messages)
response_str = "The weather in Seattle is currently 15°C and partly cloudy with light winds from the northwest."

tool_definitions_str = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "query": query_str,
            "response": response_str,
            "tool_definitions": tool_definitions_str
        }
    )
)

#### Tool Definitions as Single Dict

In [ ]:
# Test Case 6: Tool definitions as single dict
query_dict = "Check the weather in Seattle"
response_dict = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_456",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_456",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "14°C", "condition": "rainy"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The current weather in Seattle is rainy with a temperature of 14°C.",
            }
        ],
    }
]

# Tool definition as a single dict (not a list)
tool_definition_dict = [
    {
    "name": "fetch_weather",
    "description": "Fetches the weather information for the specified location.",
    "parameters": {
        "type": "object",
        "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
    },
}
]

# Append to evaluation_contents
evaluation_contents.append(
    SourceFileContentContent(
        item={
            "query": query_dict,
            "response": response_dict,
            "tool_definitions": tool_definition_dict
        }
    )
)

### Run Evaluation on All Test Cases

Now that we've defined all test cases, let's run the evaluation once on all of them.

In [ ]:
results = run_evaluator(
    evaluator_name="tool_output_utilization",
    evaluation_contents=evaluation_contents,
    data_source_config=data_source_config,
    initialization_parameters=initialization_parameters,
    data_mapping=data_mapping
)

### Display Results

View the evaluation results for each test case.

In [ ]:
pprint(results)